## Creating performance curves

The following is the MIT Review article and the original on using neural nets to solve PDEs. What's relevant here is that they used the physics insight that they should Fourier transform the inputs.

https://www.technologyreview.com/2020/10/30/1011435/ai-fourier-neural-network-cracks-navier-stokes-and-partial-differential-equations/

https://arxiv.org/abs/2010.08895

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from pathlib import Path
import pickle
import itertools

import qng
import obnetwork2

In [3]:
%matplotlib inline

## Data reading and prep

In [4]:
obsim_mm_means_df = pd.read_csv('../analysis_exp9_tandem05_nodischadj/mmdata/obsim_mm_means_df.csv', index_col=0)

Create lists of columns for subsetting the main dataframe using only relevant modeling columns for each unit. Then use these lists to create the various X dataframes.

In [5]:
# Define which columns are in which matrices starting with no queueing vars
X_pp_noq_cols = ['lam_obs', 'alos_pp', 'alos_pp_noc', 'alos_pp_c',
                 'tot_c_rate', 'cap_pp']

X_ldr_noq_cols = ['lam_obs', 'alos_obs', 'alos_ldr', 'cap_ldr',
                  'alos_pp', 'alos_pp_noc', 'alos_pp_c',
                  'tot_c_rate', 'cap_pp']

X_obs_noq_cols = ['lam_obs', 'alos_obs', 'cap_obs', 'alos_ldr', 'cap_ldr',
                  'alos_pp', 'alos_pp_noc', 'alos_pp_c',
                  'tot_c_rate', 'cap_pp']

# For "basicq" matrices, only load and rho variables are added
X_pp_basicq_cols = X_pp_noq_cols.copy()
X_pp_basicq_cols.extend(['load_pp', 'rho_pp'])

X_ldr_basicq_cols = X_ldr_noq_cols.copy()
X_ldr_basicq_cols.extend(['load_ldr', 'rho_ldr', 'load_pp', 'rho_pp'])

X_obs_basicq_cols = X_obs_noq_cols.copy()
X_obs_basicq_cols.extend(['load_obs', 'rho_obs', 'load_ldr',
                          'rho_ldr', 'load_pp', 'rho_pp'])

# For "q" matrices, include additional queueing approximations (not applicable
# to PP since unaffected by upstream unit and has no downstream unit

# LDR can have LOS shortened by patients blocked in OBS and have LOS lengthened
# by patients blocked in LDR by PP
X_ldr_q_cols = X_ldr_basicq_cols.copy()
X_ldr_q_cols.extend(['prob_blockedby_pp_approx', 'condmeantime_blockedbypp_approx',
                     'prob_blockedby_ldr_approx', 'condmeantime_blockedbyldr_approx',
                     'ldr_effmean_svctime_approx'])

# OBS modeled as infinite capacity system but time in system impacted by
# congestion in the downstream units.
X_obs_q_cols = X_obs_basicq_cols.copy()
X_obs_q_cols.extend(['prob_blockedby_pp_approx', 'condmeantime_blockedbypp_approx',
                     'prob_blockedby_ldr_approx', 'condmeantime_blockedbyldr_approx',
                     'ldr_effmean_svctime_approx'])


# Create dataframes based on the column specs above
X_pp_noq = obsim_mm_means_df.loc[:, X_pp_noq_cols]
X_ldr_noq = obsim_mm_means_df.loc[:, X_ldr_noq_cols]
X_obs_noq = obsim_mm_means_df.loc[:, X_obs_noq_cols]

# PP
X_pp_basicq = obsim_mm_means_df.loc[:, X_pp_basicq_cols]
X_pp_basicq['sqrt_load_pp'] = X_pp_basicq['load_pp'] ** 0.5

# LDR
X_ldr_basicq = obsim_mm_means_df.loc[:, X_ldr_basicq_cols]
X_ldr_basicq['sqrt_load_ldr'] = X_ldr_basicq['load_ldr'] ** 0.5
X_ldr_basicq['sqrt_load_pp'] = X_ldr_basicq['load_pp'] ** 0.5

X_ldr_q = obsim_mm_means_df.loc[:, X_ldr_q_cols]
X_ldr_q['sqrt_load_ldr'] = X_ldr_q['load_ldr'] ** 0.5
X_ldr_q['sqrt_load_pp'] = X_ldr_q['load_pp'] ** 0.5

# OBS
X_obs_basicq = obsim_mm_means_df.loc[:, X_obs_basicq_cols]
X_obs_basicq['sqrt_load_obs'] = X_obs_basicq['load_obs'] ** 0.5
X_obs_basicq['sqrt_load_ldr'] = X_obs_basicq['load_ldr'] ** 0.5
X_obs_basicq['sqrt_load_pp'] = X_obs_basicq['load_pp'] ** 0.5

X_obs_q = obsim_mm_means_df.loc[:, X_obs_q_cols]
X_obs_q['sqrt_load_obs'] = X_obs_q['load_obs'] ** 0.5
X_obs_q['sqrt_load_ldr'] = X_obs_q['load_ldr'] ** 0.5
X_obs_q['sqrt_load_pp'] = X_obs_q['load_pp'] ** 0.5

# y vectors
y_pp_occ_mean = obsim_mm_means_df.loc[:, 'occ_mean_mean_pp']
y_pp_occ_p95 = obsim_mm_means_df.loc[:, 'occ_mean_p95_pp']
y_ldr_occ_mean = obsim_mm_means_df.loc[:, 'occ_mean_mean_ldr']
y_ldr_occ_p95 = obsim_mm_means_df.loc[:, 'occ_mean_p95_ldr']
y_obs_occ_mean = obsim_mm_means_df.loc[:, 'occ_mean_mean_obs']
y_obs_occ_p95 = obsim_mm_means_df.loc[:, 'occ_mean_p95_obs']

y_mean_pct_blocked_by_pp = obsim_mm_means_df.loc[:, 'prob_blockedby_pp_sim']
y_mean_pct_blocked_by_ldr = obsim_mm_means_df.loc[:, 'prob_blockedby_ldr_sim']
y_condmeantime_blockedbyldr = obsim_mm_means_df.loc[:, 'condmeantime_blockedbyldr_sim']
y_condmeantime_blockedbypp = obsim_mm_means_df.loc[:, 'condmeantime_blockedbypp_sim']

### Creating scenario dataframe to drive graph

In [6]:
arrival_rate = np.linspace(5, 15, 11)
alos_obs = np.atleast_1d(0.1)
alos_ldr = np.atleast_1d(0.5)
cap_ldr = np.atleast_1d(16)
cap_pp = np.atleast_1d(36)
alos_pp_noc = np.atleast_1d(2)
alos_pp_c = np.atleast_1d(3)
tot_c_rate = np.array([0.2, 0.25, 0.3])

In [7]:
input_scenarios = [scn for scn in itertools.product(arrival_rate, alos_obs, alos_ldr, cap_ldr,
                            alos_pp_noc, alos_pp_c, tot_c_rate, cap_pp)]

cols = ['lam_obs',
 'alos_obs',
 'alos_ldr',
 'cap_ldr',
 'alos_pp_noc',
 'alos_pp_c',
 'tot_c_rate',
 'cap_pp']

scenarios_io_df = pd.DataFrame(input_scenarios, columns=cols)


In [8]:
scenarios_io_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   lam_obs      33 non-null     float64
 1   alos_obs     33 non-null     float64
 2   alos_ldr     33 non-null     float64
 3   cap_ldr      33 non-null     int64  
 4   alos_pp_noc  33 non-null     int64  
 5   alos_pp_c    33 non-null     int64  
 6   tot_c_rate   33 non-null     float64
 7   cap_pp       33 non-null     int64  
dtypes: float64(4), int64(4)
memory usage: 2.2 KB


In [9]:
scenarios_io_df['alos_pp'] = scenarios_io_df['tot_c_rate'] * scenarios_io_df['alos_pp_c'] + (1 - scenarios_io_df['tot_c_rate']) * scenarios_io_df['alos_pp_noc']

In [10]:
scenarios_io_df['load_ldr'] = scenarios_io_df['lam_obs'] * scenarios_io_df['alos_ldr']
scenarios_io_df['rho_ldr'] = scenarios_io_df['load_ldr'] / scenarios_io_df['cap_ldr']

scenarios_io_df['load_pp'] = scenarios_io_df['lam_obs'] * scenarios_io_df['alos_pp']
scenarios_io_df['rho_pp'] = scenarios_io_df['load_pp'] / scenarios_io_df['cap_pp']

scenarios_io_df['sqrt_load_ldr'] = np.sqrt(scenarios_io_df['rho_ldr'])
scenarios_io_df['sqrt_load_pp'] = np.sqrt(scenarios_io_df['rho_pp'])

In [11]:
scenarios_io_df['pp_cv2_svctime'] = 0.12
scenarios_io_df['ldr_cv2_svctime'] = 0.5
scenarios_io_df['obs_cv2_svctime'] = 1.0

In [12]:
scenarios_io_df['prob_blockedby_pp_approx'] = scenarios_io_df.apply(lambda x: obnetwork2.ldr_prob_blockedby_pp_hat(
    x.lam_obs, x.alos_pp, x.cap_pp, x.pp_cv2_svctime), axis=1)

In [13]:
scenarios_io_df['condmeantime_blockedbypp_approx'] = scenarios_io_df.apply(lambda x: obnetwork2.ldr_condmeantime_blockedby_pp_hat(
    x.lam_obs, x.alos_pp, x.cap_pp, x.pp_cv2_svctime), axis=1)

In [14]:
scenarios_io_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   lam_obs                          33 non-null     float64
 1   alos_obs                         33 non-null     float64
 2   alos_ldr                         33 non-null     float64
 3   cap_ldr                          33 non-null     int64  
 4   alos_pp_noc                      33 non-null     int64  
 5   alos_pp_c                        33 non-null     int64  
 6   tot_c_rate                       33 non-null     float64
 7   cap_pp                           33 non-null     int64  
 8   alos_pp                          33 non-null     float64
 9   load_ldr                         33 non-null     float64
 10  rho_ldr                          33 non-null     float64
 11  load_pp                          33 non-null     float64
 12  rho_pp                  

In [15]:
scenarios_io_df['prob_blockedby_ldr_approx'] = \
    input_scenarios_df.apply(lambda x: obnetwork2.obs_blockedby_ldr_hats(
        x.lam_obs, x.tot_c_rate, x.alos_ldr, x.ldr_cv2_svctime, x.cap_ldr, 
    x.alos_pp, x.pp_cv2_svctime, x.cap_pp)[2], axis=1)

In [16]:
scenarios_io_df['condmeantime_blockedbyldr_approx'] = \
    input_scenarios_df.apply(lambda x: obnetwork2.obs_blockedby_ldr_hats(
        x.lam_obs, x.tot_c_rate, x.alos_ldr, x.ldr_cv2_svctime, x.cap_ldr, 
    x.alos_pp, x.pp_cv2_svctime, x.cap_pp)[3], axis=1)

In [17]:
scenarios_io_df['ldr_effmean_svctime_approx'] = \
    input_scenarios_df.apply(lambda x: obnetwork2.obs_blockedby_ldr_hats(
        x.lam_obs, x.tot_c_rate, x.alos_ldr, x.ldr_cv2_svctime, x.cap_ldr, 
    x.alos_pp, x.pp_cv2_svctime, x.cap_pp)[1], axis=1)

Now just want the columns that exist in `X_ldr_q` to use in making predictions for the different perf measures with the various models we already fit with sklearn. In addition, we want to compare with some direct queueing approximations.

In [30]:
X_ldr_q_scenarios_df = scenarios_io_df.loc[:, X_ldr_q_cols]

In [ ]:
ldr_occ_mean_direct_approx = X_ldr_q['lam_obs'] * X_ldr_q['ldr_effmean_svctime_approx'] /24.0 

In [ ]:
prediction_scatter(y_ldr_occ_mean, ldr_occ_mean_direct_approx)

In [ ]:
ldr_occ_p95_direct_approx = ldr_occ_mean_direct_approx + 1.645 * np.sqrt(ldr_occ_mean_direct_approx)

In [ ]:
prediction_scatter(y_ldr_occ_p95, ldr_occ_p95_direct_approx)

For the over predictions, perhaps we are bumping up against capacity restrictions. A simple metametamodel that includes the direct approximation along with rho_ldr might do ok.

In [ ]:
X_ldr_approx = pd.DataFrame({'ldr_occ_p95_direct_approx': ldr_occ_p95_direct_approx,
               'rho_ldr': X_ldr_q.loc[:, 'rho_ldr']})

In [ ]:
X_ldr_approx

In [ ]:
ldr_occ_p95_q_approx_results = fit_summarize_mm('ldr_occ_p95_q_approx', X_ldr_approx, y_ldr_occ_p95, 
                                                 scale=False, fit_intercept=True, 
                                                 flavor='lm')
ldr_occ_p95_q_approx_results['coeffs_df']
